# Agent

Agents are autonomous decision-making and execution processes, with the core being the use of llm as a reasoning engine. Based on llm's understanding of tasks and environments, and with various tools provided, they autonomously decide on a series of actions.

In [5]:
import { load } from "dotenv";
const env = await load();

const process = {
    env
}

In [6]:
import { z } from "zod";
import { ChatOpenAI } from "@langchain/openai";
import { JsonOutputToolsParser } from "@langchain/core/output_parsers/openai_tools";
import { RunnableSequence, RunnableBranch, RunnablePassthrough } from "@langchain/core/runnables";
import { zodToJsonSchema } from "zod-to-json-schema";
import { ChatPromptTemplate, PromptTemplate } from "@langchain/core/prompts";
 
const classifySchema = z.object({
    type: z.enum(["科普", "编程", "一般问题"]).describe("用户提问的分类")
})

const model = new ChatOpenAI({
    temperature: 0 
})

const modelWithTools = model.bind({
    tools: [
        {
            type: "function",
            function: {
                name: "classifyQuestion",
                description: "对用户的提问进行分类",
                parameters: zodToJsonSchema(classifySchema),
            }
        }
    ],
    tool_choice: {
        type: "function",
        function: {
           name: "classifyQuestion"
        }
    }
})

const prompt = ChatPromptTemplate.fromMessages([
    ["system", `仔细思考，你有充足的时间进行严谨的思考，然后对用户的问题进行分类，
    当你无法分类到特定分类时，可以分类到 "一般问题"`],
    ["human", "{input}"]
])

const classifyChain = RunnableSequence.from([
    prompt,
    modelWithTools,
    new JsonOutputToolsParser(),
    (input) => {
        const type = input[0]?.args?.type
        return type ? type : "一般问题"
    }
])

In [7]:
await classifyChain.invoke({
    "input": "鲸鱼是哺乳动物么？"
})

"科普"

In [8]:
import { StringOutputParser } from "@langchain/core/output_parsers";

const answeringModel = new ChatOpenAI({
    temperature: 0.7,
})

const sciencePrompt = PromptTemplate.fromTemplate(
  `作为一位科普专家，你需要解答以下问题，尽可能提供详细、准确和易于理解的答案：

问题：{input}
答案：`
)
    
const programmingPrompt = PromptTemplate.fromTemplate(
  `作为一位编程专家，你需要解答以下编程相关的问题，尽可能提供详细、准确和实用的答案：

问题：{input}
答案：`
)

const generalPrompt = PromptTemplate.fromTemplate(
  `请回答以下一般性问题，尽可能提供全面和有深度的答案：

问题：{input}
答案：`
)


const scienceChain = RunnableSequence.from([
    sciencePrompt,
    answeringModel,
    new StringOutputParser(),
    {
        output: input => input,
        role: () => "科普专家"
    }
    
])

const programmingChain = RunnableSequence.from([
    programmingPrompt,
    answeringModel,
    new StringOutputParser(),
    {
        output: input => input,
        role: () => "编程大师"
    }
    
])

const generalChain = RunnableSequence.from([
    generalPrompt,
    answeringModel,
    new StringOutputParser(),
    {
        output: input => input,
        role: () => "通识专家"
    }
    
])

In [9]:
const branch = RunnableBranch.from([
    [
      (input => input.type.includes("科普")),
      scienceChain,
    ],
    [
      (input => input.type.includes("编程")),
      programmingChain,
    ],
    generalChain
  ]);

In [10]:
const route = ({ type }) => {
    if(type.includes("科普")){
        return scienceChain
    }else if(type.includes("编程")){
        return programmingChain
    }

    return generalChain
}

In [11]:
const outputTemplate = PromptTemplate.fromTemplate(
    `感谢您的提问，这是来自 {role} 的专业回答：
    
    {output}
    `)
    
    
    const finalChain = RunnableSequence.from([
        {
            type: classifyChain,
            input: input => input.input
        },
        branch,
        (input) => outputTemplate.format(input),
    ])

In [12]:
const res = await finalChain.invoke({
    "input": "鲸鱼是哺乳动物么？"
})

console.log(res)

感谢您的提问，这是来自 科普专家 的专业回答：
    
    是的，鲸鱼是哺乳动物。哺乳动物是一类具有特殊特征的动物，它们能够产生乳汁喂养幼崽。鲸鱼具有哺乳动物的所有特征，包括体内产生乳汁、具有毛发、体温恒定和直接生育等。鲸鱼的哺乳方式是通过乳头喂养幼崽，使它们得到营养和生长所需的物质。此外，鲸鱼还具有其他哺乳动物的特点，如具有适应水生生活的特殊结构和行为。因此，我们可以确定鲸鱼属于哺乳动物。
    
